In [ ]:
    PROJECT_ID = "cloud-llm-preview1"  # @param {type:"string"}
    LOCATION = "us-central1"  # @param {type:"string"}

    # Initialize Vertex AI
    import vertexai

    vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
import time
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)

In [ ]:
multimodal_model = GenerativeModel("gemini-ultra-vision")
text_model= GenerativeModel("gemini-ultra")

In [ ]:
contents="You are a data steward. Your role is to produce human meaningful metadata descriptions.\
Table's fully qualified name:bigquery:jsk-dataplex-demo-380508.metadata_generation.cc \
These SQL queries can be used to generate the data in table \
1. insert into metadata_generation.cc select * from metadata_generation.crimes;\
2. insert into metadata_generation.cc SELECT unique_key uk, case_number cn, date as dt, block b, iucr as i, primary_type as pt, description as de, location_description as ld, arrest as ar, domestic as d, beat as bt, district di, ward as wr, community_area as cs, fbi_code as fc, x_coordinate xc, y_coordinate yc, year yr, updated_on uo, latitude la, longitude lo, location ltn FROM `bigquery-public-data.chicago_crime.crime` ; \
3. create or replace table metadata_generation.cc as SELECT unique_key uk, case_number cn, date as dt, block b, iucr as i, primary_type as pt, description as de, location_description as ld, arrest as ar, domestic as d, beat as bt, district di, ward as wr, community_area as cs, fbi_code as fc, x_coordinate xc, y_coordinate yc, year yr, updated_on uo, latitude la, longitude lo, location ltn FROM `bigquery-public-data.chicago_crime.crime` ;\
4. create or replace table metadata_generation.cc as SELECT unique_key uk, case_number cn, date as dt, block b, iucr as i, primary_type as pt, description as de, location_description as ld, arrest as ar, domestic as d, beat as bt, district di, ward as wr, community_area as cs, fbi_code as fc, x_coordinate xc, y_coordinate yc, year yr, updated_on uo, latitude la, longitude lo, location ltn FROM `bigquery-public-data.chicago_crime.crime` LIMIT 1000 ;\

"

In [ ]:
responses = text_model.generate_content(contents,stream=True)

print("\n-------Response--------")


In [ ]:

for response in responses:
    print(response.text, end="")

### Let's list all tables we want to analyze


In [ ]:
table_list=[
    {'dataset': 'iowa_liquor_sales', 'table': 'sales'},
    {'dataset': 'london_bicycles', 'table': 'cycle_hire'},
    {'dataset': 'london_bicycles', 'table': 'cycle_stations'},
    {'dataset': 'ml_datasets', 'table': 'census_adult_income'},
    {'dataset': 'ml_datasets', 'table': 'credit_card_default'},
    {'dataset': 'ml_datasets', 'table': 'holidays_and_events_for_forecasting'},
    {'dataset': 'ml_datasets', 'table': 'iris'},
    {'dataset': 'ml_datasets', 'table': 'penguins'},
    {'dataset': 'ml_datasets', 'table': 'ulb_fraud_detection'},
    {'dataset': 'new_york_citibike', 'table': 'citibike_stations'},
    {'dataset': 'new_york_citibike', 'table': 'citibike_trips'},
    {'dataset': 'fdic_banks', 'table': 'institutions'},
    {'dataset': 'fdic_banks', 'table': 'locations'},
    {'dataset': 'fda_food', 'table': 'food_enforcement'},
    {'dataset': 'fda_food', 'table': 'food_events'},
    {'dataset': 'fcc_political_ads', 'table': 'broadcast_tv_radio_station'},
    {'dataset': 'fcc_political_ads', 'table': 'content_info'},
    {'dataset': 'fcc_political_ads', 'table': 'file_history'},
    {'dataset': 'fcc_political_ads', 'table': 'file_record'},
]

### Let's get schema for the table

In [ ]:
from google.cloud import bigquery
import requests
from google.oauth2 import service_account
import vertexai
from vertexai.language_models import TextGenerationModel
from google.cloud import datacatalog_lineage_v1 as lineage_v1


def _get_table_schema_bq(project_id, dataset_name, table_name):
    bqclient = bigquery.Client(project=project_id)
    """Get the schema of a table in BigQuery."""
    table = bqclient.get_table(f"{dataset_name}.{table_name}")
    return table.schema

print (_get_table_schema_bq('bigquery-public-data','ml_datasets','iris')) 

project_id='cloud-llm-preview1'

In [ ]:
def _get_upstream_tables(project_id,dataset,tablename,debug=0):
    # Create a client
    client = lineage_v1.LineageClient()
    target = lineage_v1.EntityReference()
    #source.fully_qualified_name = "bigquery:bigquery-public-data.ml_datasets.iris"
    target = lineage_v1.EntityReference()
    target.fully_qualified_name = "bigquery:"+project_id+"."+dataset+"."+tablename
    if debug==1:
        print("fully_qualified_name:"+target.fully_qualified_name)
    # Initialize request argument(s)
    request = lineage_v1.SearchLinksRequest(
        parent="projects/446454295341/locations/us",
        target=target,
    )

    # Make the request
    page_result = client.search_links(request=request)

    bigquery_tables=[]
    # Handle the response
    for response in page_result:
        if str(response.source.fully_qualified_name).startswith("bigquery:"):
            bigquery_tables.append(str(response.source.fully_qualified_name).replace("bigquery:",""))
        if debug==1:
            for x in bigquery_tables:
                print("substrings with names:"+x)
    return bigquery_tables


#sample_list_links()
tablesy=_get_upstream_tables('jsk-dataplex-demo-380508','taxi_data','taxi_trips',1)

print(tablesy)

In [313]:
#_get_table_schema('ml_datasets','iris')
import google.cloud.dataplex_v1 as dplx
import json
from google.protobuf.json_format import MessageToJson


dplx_client=dplx.DataScanServiceClient()


def _get_single_data_scanname(tableref):
    
    response=dplx_client.list_data_scans(parent="projects/jsk-dataplex-demo-380508/locations/us-central1")
    #print(response)
    for element in response:
        print("START HERE---->\n"+str(element))
        
        print(element.data.resource)
        print("element resource name:"+element.data.resource)
        print("string position:"+str(str(element.data.resource).find(tableref)))
        if str(element.data.resource).find(tableref) > -1:
            
            print("FOUND at ->"+str(str(element.data.resource).find(tableref))+"\n element name:"+element.name)
            print("END HERE---->\n")
            return element.name
        else: 
            print("NOT FOUND in :"+str(element.data.resource))
        print("END HERE---->\n")
        
#test='//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/metadata_generation/tables/cc'
#print(test.find('cc'))     
output=_get_single_data_scanname('cc')


START HERE---->
name: "projects/jsk-dataplex-demo-380508/locations/us-central1/dataScans/simple-scan"
uid: "19f630a2-3a94-427a-be62-77ab215ae133"
display_name: "simple scan"
state: ACTIVE
create_time {
  seconds: 1707298498
  nanos: 605295541
}
update_time {
  seconds: 1707298504
  nanos: 21725447
}
data {
  resource: "//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tables/taxi_trips"
}
execution_spec {
  trigger {
    on_demand {
    }
  }
}
execution_status {
  latest_job_start_time {
    seconds: 1707298542
    nanos: 220000000
  }
  latest_job_end_time {
    seconds: 1707298573
    nanos: 872696270
  }
}
type_: DATA_PROFILE

//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tables/taxi_trips
element resource name://bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tables/taxi_trips
string position:-1
NOT FOUND in ://bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tabl

TypeError: can only concatenate str (not "int") to str

In [284]:
#dplx_client.list_data_scans(parent="projects/jsk-dataplex-demo-380508/locations/us-central1")

def _get_table_profile(tablename):

        scanname=_get_single_data_scanname(tablename)


        requestscan=dplx.GetDataScanRequest(name=scanname)
        response_scan = dplx_client.get_data_scan(request=requestscan)
        # Set a GetDataScanRequest() using the args.datascan_name argument.

        dq_scan_name = response_scan.name
        jobs_request = dataplex_v1.ListDataScanJobsRequest(
                parent=scanname,
                page_size=10)
                # optional: filter, page_size

        page_result = dplx_client.list_data_scan_jobs(request=jobs_request)

        counter = 0
        job_names = []
        for response in page_result:
                counter += 1
                job_names.append(response.name)

        print('Jobs scanned: ' + str(counter))


        for job_name in job_names:
                job_request = dataplex_v1.GetDataScanJobRequest(
                        name=job_name,
                        view="FULL",
                )

                job_result = dplx_client.get_data_scan_job(request=job_request)
                        # Skips jobs if not in succeeded state
                if job_result.state != 4:
                        continue

                dq_job_id = job_result.uid

                print("dq_scan_name --> " + dq_scan_name)
                print("dq_job_id --> " + dq_job_id)
                print("job_result.data_quality_result.row_count --> " + str(job_result.data_quality_result.row_count))
                print("job_result.start_time --> " + str(job_result.start_time))
                print("job_result.end_time --> " + str(job_result.end_time))
                print("MessageToJson(job_result.data_profile_result.scanned_data._pb) --> " + MessageToJson(job_result.data_profile_result.scanned_data._pb))

                profile=MessageToJson(job_result.data_profile_result.profile._pb)
                return profile

                
profil=_get_table_profile('cc')
        

AttributeError: 'dict' object has no attribute 'data'

In [168]:
project_id = 'jsk-dataplex-demo-380508'

def build_context():
    profil=_get_table_profile('crimes')
    table_description=_get_table_bq_metadata('jsk-dataplex-demo-380508.metadata_generation.crimes')
    load_query=get_process_details('projects/446454295341/locations/us/processes/e8042f025185164c319acb1272629893')

    print("--------------Printing retrieved metadata: --------------")
    #print(profil)
    print(table_description.)
    #print(load_query)
build_context()

found at ->-1
 element name:projects/jsk-dataplex-demo-380508/locations/us-central1/dataScans/a77b3de83-e198-40e1-921a-d322c6e46eca
Jobs scanned: 1
dq_scan_name --> projects/jsk-dataplex-demo-380508/locations/us-central1/dataScans/a77b3de83-e198-40e1-921a-d322c6e46eca
dq_job_id --> 0129f297-acd6-49be-b9cb-47c2707a301f
job_result.data_quality_result.row_count --> 0
job_result.start_time --> 2024-02-05 11:10:45.121000+00:00
job_result.end_time --> 2024-02-05 11:11:12.330388+00:00
MessageToJson(job_result.data_profile_result.scanned_data._pb) --> {}
jsk-dataplex-demo-380508.metadata_generation.crimes None {'dataplex-dp-published-scan': 'a77b3de83-e198-40e1-921a-d322c6e46eca', 'dataplex-dp-published-project': 'jsk-dataplex-demo-380508', 'dataplex-dp-published-location': 'us-central1'}
US:bquxjob_fbd9c05_18d78ec937e
Type: query
State: DONE
Created: 2024-02-05T10:59:47.151000+00:00
Query: create or replace table metadata_generation.cc as
SELECT unique_key uk, 
case_number cn, 
date as dt, 
b

In [164]:
def _get_table_bq_metadata(table_ref):
    bqclient = bigquery.Client(project=project_id)
    table = bqclient.get_table(table_ref)

    print(table,table.description,table.labels)
    return(table)

_get_table_bq_metadata('bigquery-public-data.ml_datasets.iris')
print(_get_table_bq_metadata('jsk-dataplex-demo-380508.taxi_data.source_table'))

NameError: name 'project_id' is not defined

In [ ]:
profile_json=json.loads(profile)
for i in profile_json['fields']:
    print(i['name'])
    print(i['type'])
    print(i['mode'])
    print(i['profile'])


In [186]:
from  google.cloud import datacatalog_v1


def _search_data_catalog(phrase):
    dc_client=datacatalog_v1.DataCatalogClient()
    project_id='jsk-dataplex-demo-380508'
    search_string = phrase
    scope = datacatalog_v1.types.SearchCatalogRequest.Scope()
    scope.include_project_ids.append(project_id)
    search_results = dc_client.search_catalog(scope=scope, query=search_string)

    results_list = []
    for result in search_results:
        result_dict = json.loads(MessageToJson(result._pb))
        result_dict['image_url'] = "https://lh3.googleusercontent.com/p9ST3mhfKqDdxwwgyGHCFmCddgFeHnYlQfCbORDHJm48z1cZhEknPXlbY_iGsnr2sIPk8EVanoqGjA=s48-w48-rw-lo" # Add the image_url to the dictionary
        results_list.append(result_dict)
        #results_list.append(result)
    return results_list

output=_search_data_catalog('taxi')


In [189]:
for i in output:
    #print(i._pb)
    print(i)
    break

{'searchResultType': 'ENTRY', 'searchResultSubtype': 'entry.table', 'relativeResourceName': 'projects/jsk-dataplex-demo-380508/locations/us/entryGroups/@bigquery/entries/cHJvamVjdHMvanNrLWRhdGFwbGV4LWRlbW8tMzgwNTA4L2RhdGFzZXRzL3RheGlfZGF0YS90YWJsZXMvdGF4aV90cmlwcw', 'linkedResource': '//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tables/taxi_trips', 'modifyTime': '2024-02-02T08:50:21Z', 'integratedSystem': 'BIGQUERY', 'fullyQualifiedName': 'bigquery:jsk-dataplex-demo-380508.taxi_data.taxi_trips', 'image_url': 'https://lh3.googleusercontent.com/p9ST3mhfKqDdxwwgyGHCFmCddgFeHnYlQfCbORDHJm48z1cZhEknPXlbY_iGsnr2sIPk8EVanoqGjA=s48-w48-rw-lo'}


In [200]:
def _get_entry_details(entry_id):
     dc_client=datacatalog_v1.DataCatalogClient()
     request=datacatalog_v1.LookupEntryRequest(linked_resource = entry_id)
     results=dc_client.lookup_entry(request=request)
 #   entry = dc_client.lookup_entry(request={"linked_resource": entry_id})
 #   entry_dict = json.loads(MessageToJson(entry._pb))
     return results

output=_get_entry_details('//bigquery.googleapis.com/projects/jsk-dataplex-demo-380508/datasets/taxi_data/tables/taxi_trips')

In [201]:
field_json=MessageToJson(output._pb)
print(MessageToJson(output._pb))

{
  "name": "projects/jsk-dataplex-demo-380508/locations/us/entryGroups/@bigquery/entries/cHJvamVjdHMvanNrLWRhdGFwbGV4LWRlbW8tMzgwNTA4L2RhdGFzZXRzL3RheGlfZGF0YS90YWJsZXMvdGF4aV90cmlwcw",
  "type": "TABLE",
  "schema": {
    "columns": [
      {
        "type": "STRING",
        "description": "Test",
        "mode": "NULLABLE",
        "column": "TaxiCompany"
      },
      {
        "type": "INT64",
        "description": "Dupa",
        "mode": "NULLABLE",
        "column": "Vendor_Id"
      },
      {
        "type": "TIMESTAMP",
        "description": "why not",
        "mode": "NULLABLE",
        "column": "Pickup_DateTime"
      },
      {
        "type": "TIMESTAMP",
        "description": "Tu tez cos jest",
        "mode": "NULLABLE",
        "column": "Dropoff_DateTime"
      },
      {
        "type": "STRING",
        "mode": "NULLABLE",
        "column": "Store_And_Forward"
      },
      {
        "type": "INT64",
        "mode": "NULLABLE",
        "column": "Rate_Code_Id

In [ ]:
def _get_table _dataplex_profile(table_ref):
    bqclient = bigquery.Client(project=project_id)
    table = bqclient.get_table(table_ref)

    print(table,table.description,table.labels)
    return(table)1

In [ ]:
from google.cloud import datacatalog_lineage_v1

def sample_list_processes():
    # Create a client
    client = datacatalog_lineage_v1.LineageClient()

    # Initialize request argument(s)
    request = datacatalog_lineage_v1.ListProcessesRequest(
        parent="projects/446454295341/locations/us",

    )

    # Make the request
    page_result = client.list_processes(request=request)

    # Handle the response
    for response in page_result:
        print(response)
        print(MessageToJson(response._pb))
        
sample_list_processes()

In [ ]:
from google.cloud import bigquery


def _bq_job_details(job_name):
    client = bigquery.Client
    location = "us"
    job_id = "bquxjob_35b0569c_18d59fa2d41"

    bqclient=bigquery.Client(project='jsk-dataplex-demo-380508')
 

    job = client.get_job(bqclient,job_id=job_id, location=location)

    # All job classes have "location" and "job_id" string properties.
    # Use these properties for job operations such as "cancel_job" and
    # "delete_job".
    print(f"{job.location}:{job.job_id}")
    print(f"Type: {job.job_type}")
    print(f"State: {job.state}")
    print(f"Created: {job.created.isoformat()}")
    print(f"Query: {job.query}")

_bq_job_details('bquxjob_35b0569c_18d59fa2d41')

In [ ]:
from google.cloud import datacatalog_lineage_v1

def sample_list_runs():
    # Create a client
    client = datacatalog_lineage_v1.LineageClient()

    # Initialize request argument(s)
    request = datacatalog_lineage_v1.ListRunsRequest(
        parent="projects/446454295341/locations/us/processes/64d98e962706f0223f3e957fedf5af2b-73c9499597cc075e4716c4ecfeb5c058",

    )

    # Make the request
    page_result = client.list_runs(request=request)

    # Handle the response
    for response in page_result:
        print(response)
        print(MessageToJson(response._pb)
        
sample_list_runs()

In [ ]:
def sample_list_events():
    # Create a client
    client = datacatalog_lineage_v1.LineageClient()

    # Initialize request argument(s)
    request = datacatalog_lineage_v1.ListLineageEventsRequest(
        parent="projects/446454295341/locations/us/processes/64d98e962706f0223f3e957fedf5af2b-73c9499597cc075e4716c4ecfeb5c058/runs/2024-01-04T11:34:02.440095Z",

    )

    # Make the request
    page_result = client.list_lineage_events(request=request)

    # Handle the response
    for response in page_result:
        print(response)
        print(MessageToJson(response._pb))
        
sample_list_events()


In [275]:
from google.cloud import dataplex_v1 #import lineage_v1
from google.cloud import datacatalog_lineage_v1 as lineage_v1


def lineage_source_tables(target_fqdn):
    # Create a client
    client = lineage_v1.LineageClient()
    target = lineage_v1.EntityReference()
    #source.fully_qualified_name = "bigquery:bigquery-public-data.ml_datasets.iris"
    target = lineage_v1.EntityReference()
    target.fully_qualified_name = target_fqdn

    # Initialize request argument(s)
    request = lineage_v1.SearchLinksRequest(
        parent="projects/446454295341/locations/us",
        target=target,
    )

    print('lineage_source_tables: looking for lineage links for table:'+target_fqdn)
    # Make the request
    page_result = client.search_links(request=request)
    response_list=[]
  
    # Handle the response
    for response in page_result:
        new_element={}
        new_element['name']=response.name
        new_element['source']=response.source.fully_qualified_name
        new_element['target']=response.target.fully_qualified_name
        print('lineage_source_tables: found lineage link:'+response.name)
        response_list.append(new_element)

   # print(json.loads(MessageToJson(page_result._pb)))
    return MessageToJson(page_result._pb)

def find_lineage_processes(target_table):
    #create a client
    client = lineage_v1.LineageClient()
    #client.batch_search_link_processes()
    #initialize request arguments
    print('find_lineage_processes: looking for lineage links for table:'+target_table)
    response=json.loads(lineage_source_tables(target_table))
    print('find_lineage_processes: response->'+json.dumps(response))
    links=[]
    for i in response['links']:
        links.append(i['name'])
        print('find_lineage looking for link:' + i['name'])
    request = lineage_v1.BatchSearchLinkProcessesRequest(parent="projects/446454295341/locations/us",links=links)
    #make the request

    page_result = client.batch_search_link_processes(request=request)
    return MessageToJson(page_result._pb)
    
#find_lineage_processes("bigquery:jsk-dataplex-demo-380508.taxi_data.taxi_trips")
#output=lineage_source_tables("bigquery:jsk-dataplex-demo-380508.metadata_generation.cc")
second_output=find_lineage_processes("bigquery:jsk-dataplex-demo-380508.metadata_generation.cc")
print(second_output)

find_lineage_processes: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.cc
lineage_source_tables: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.cc
lineage_source_tables: found lineage link:projects/446454295341/locations/us/links/p:f761823fd78e615f5209fe3279279c4b
lineage_source_tables: found lineage link:projects/446454295341/locations/us/links/p:59317e61604dae41b426b68d79a5eff5
find_lineage_processes: response->{"links": [{"name": "projects/446454295341/locations/us/links/p:f761823fd78e615f5209fe3279279c4b", "source": {"fullyQualifiedName": "bigquery:jsk-dataplex-demo-380508.metadata_generation.crimes"}, "target": {"fullyQualifiedName": "bigquery:jsk-dataplex-demo-380508.metadata_generation.cc"}, "startTime": "2024-02-06T21:24:11.877Z", "endTime": "2024-02-06T21:24:16.209Z"}, {"name": "projects/446454295341/locations/us/links/p:59317e61604dae41b426b68d79a5eff5", "source": {"fullyQualifiedName": "big

In [278]:
def get_dependencies_info(tablename):
    processes=[]
    
    response=json.loads(find_lineage_processes(tablename))
    for i in response['processLinks']:

        print('found process through lineage: '+i['process'])
        processes.append(i['process'])

    job_details=[]    
    for i in processes:
        result=get_process_details(i)
        job_details.append(result)
    return job_details

get_dependencies_info('bigquery:jsk-dataplex-demo-380508.metadata_generation.cc')

find_lineage_processes: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.cc
lineage_source_tables: looking for lineage links for table:bigquery:jsk-dataplex-demo-380508.metadata_generation.cc
lineage_source_tables: found lineage link:projects/446454295341/locations/us/links/p:f761823fd78e615f5209fe3279279c4b
lineage_source_tables: found lineage link:projects/446454295341/locations/us/links/p:59317e61604dae41b426b68d79a5eff5
find_lineage_processes: response->{"links": [{"name": "projects/446454295341/locations/us/links/p:f761823fd78e615f5209fe3279279c4b", "source": {"fullyQualifiedName": "bigquery:jsk-dataplex-demo-380508.metadata_generation.crimes"}, "target": {"fullyQualifiedName": "bigquery:jsk-dataplex-demo-380508.metadata_generation.cc"}, "startTime": "2024-02-06T21:24:11.877Z", "endTime": "2024-02-06T21:24:16.209Z"}, {"name": "projects/446454295341/locations/us/links/p:59317e61604dae41b426b68d79a5eff5", "source": {"fullyQualifiedName": "big

[name: "projects/446454295341/locations/us/processes/6a02b6b0bb60e248ec976390f1eb0960"
 display_name: "Query"
 attributes {
   key: "bigquery_job_id"
   value {
     string_value: "bquxjob_33997b3c_18d804e98fb"
   }
 }
 origin {
   source_type: BIGQUERY
   name: "jsk-dataplex-demo-380508:us.bquxjob_33997b3c_18d804e98fb"
 },
 name: "projects/446454295341/locations/us/processes/460ff7a90c353ea0a4ace3e5f0301cdb"
 display_name: "Query"
 attributes {
   key: "bigquery_job_id"
   value {
     string_value: "bquxjob_d7f14f1_18d80451cf8"
   }
 }
 origin {
   source_type: BIGQUERY
   name: "jsk-dataplex-demo-380508:us.bquxjob_d7f14f1_18d80451cf8"
 },
 name: "projects/446454295341/locations/us/processes/ea1975c3dcf35d4d1483166d80a1e70b"
 display_name: "Query"
 attributes {
   key: "bigquery_parent_job_id"
   value {
     string_value: "bquxjob_65b4eabe_18d78f45c17"
   }
 }
 attributes {
   key: "bigquery_job_id"
   value {
     string_value: "script_job_748f0c7e7d40117a5b901afb44d86710_0"
   }
 

In [280]:
from google.cloud import bigquery


def _bq_job_details(job_name):
    client = bigquery.Client
    location = "us"
    job_id = job_name#"bquxjob_35b0569c_18d59fa2d41"

    bqclient=bigquery.Client(project='jsk-dataplex-demo-380508')
 

    job = client.get_job(bqclient,job_id=job_id, location=location)

    # All job classes have "location" and "job_id" string properties.
    # Use these properties for job operations such as "cancel_job" and
    # "delete_job".
    print(f"{job.location}:{job.job_id}")
    print(f"Type: {job.job_type}")
    print(f"State: {job.state}")
    print(f"Created: {job.created.isoformat()}")
    print(f"Query: {job.query}")
    return job


def get_process_details (process_name):
    # Create a client
    client = datacatalog_lineage_v1.LineageClient()

    # Initialize request argument(s)
    request = datacatalog_lineage_v1.GetProcessRequest(
        name=process_name,
    )

    # Make the request
    response = client.get_process(request=request)
   # print(response)
    jsonized=json.loads(MessageToJson(response._pb))
    bq_sql=_bq_job_details(jsonized['attributes']['bigquery_job_id'])
    print("bq_sql.query:"+bq_sql.query)

    
   # if response.attributes[key=="bigquery_job_id":
   #     print(response.attributes.value.string_value)
    # Handle the response
   # print(response)
  #  print(MessageToJson(response._pb))
    return bq_sql

get_process_details('projects/446454295341/locations/us/processes/807c39109ee0a29b59bf5b6bc96dfdb8')

US:bquxjob_35b0569c_18d59fa2d41
Type: query
State: DONE
Created: 2024-01-30T10:46:24.851000+00:00
Query: insert into jsk-dataplex-demo-380508.taxi_data.taxi_trips select * from jsk-dataplex-demo-380508.taxi_data.source_table
bq_sql.query:insert into jsk-dataplex-demo-380508.taxi_data.taxi_trips select * from jsk-dataplex-demo-380508.taxi_data.source_table


QueryJob<project=jsk-dataplex-demo-380508, location=US, id=bquxjob_35b0569c_18d59fa2d41>